# <a id='toc1_'></a>[Optics notebook](#toc0_)

17.02.2025 -> 11.05.2025 - Dominique Humbert
Initial version for function building.


**Table of contents**<a id='toc0_'></a>    
- [Optics notebook](#toc1_)    
- [Inputs](#toc2_)    
  - [PSD](#toc2_1_)    
  - [Phase screen & PSF](#toc2_2_)    
- [Long exposure PSF](#toc3_)    
- [Lenslet array illumination](#toc4_)    
- [SHWFS](#toc5_)    
  - [Lenslet illumination mask](#toc5_1_)    
  - [Wavefront slope](#toc5_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
from ows import fouriertransform as mathft
from astropy.io import fits

import numpy as np
from ows import ows
import time

rad2asec = 3600 * 180/np.pi
asec2rad = 1/rad2asec


# <a id='toc2_'></a>[Inputs](#toc0_)

In [ ]:
# Input
SAVE = False
dimmat = 256
dxp = 0.00304 # Pupil plane pixel size
r0 = .1
L0 = 27
D = .1
wl = 500e-9 # [m]

SEED = None
F = 8e-3 #[m]
N = 8
PD = [dimmat//4,0]
n = 10            # Qty of images

## <a id='toc2_1_'></a>[PSD](#toc0_)
Power spectrum density

In [ ]:
# Computing limit values
FoV = 25*asec2rad # arcsec
#Npx = 1000 # image plane pixels qty
#Daf = asec2rad*FoV#/Npx # [rad/px]  (Numerical aperture)
Daf = wl/(2*D) # max
dfp = Daf/wl # [/m] Pupil plane spatial frequency pixel size (Numerical aperture/WL)
dimpsf = 8*wl*np.sqrt((D/r0)**2 + 1)/(Daf*D)   # (min psf size) Matrix size
print("Miminum matrix dimention :",dimpsf)
# dimmat = int(dimpsf)+int((dimpsf %1 !=0)*1)
# dimmat += int((dimmat %2 != 0)*1)

if dimmat < dimpsf:
    print(f"dimmat is too small: {dimmat:d}. It souhld be greater than {dimpsf:.1f}")


PSD = ows.psd(dimmat, dxp, r0, wl)
# plt.close(1)
# plt.figure(1)
# plt.imshow(PSD**(1/8))
# plt.title("Phase spectrum density ^(1/8)")
# plt.colorbar()

## <a id='toc2_2_'></a>[Phase screen & PSF](#toc0_)

In [ ]:
start = time.time()

# Prepare seeing limited psf for normalization. It could be done inside ows.phase_screen() with the OTF directly but is not in order not to compute it every time
psf_normalization_factor = ows.compute_diffLim_psf(PD, 128, dxp).max() # could also be done with telescope_otf()


psf_stack = np.zeros((PD[0],PD[0]))
before = time.time()
status = 0

for i in range(n):
    [phase_screen, psf, pupil_mask, R] = ows.phase_screen(PSD, dxp, SEED, PUPIL = True, PD = [PD[0], PD[1]])
    [lightfield, DwDx, DwDy] = ows.SHWFS(dimmat, PD, N, F, wl, pupil_mask, R, phase_screen, dxp)
    S = psf[PD[0]//2,PD[0]//2]/psf_normalization_factor
    psf_stack += ows.normalize(psf)*S
    psf = ows.normalize(psf)*S

    lightfield = S*ows.normalize(lightfield)
    # Approximation: normalize the lightfield and scale with the psf strehl ratio.
    # !!! should normalize each "PSFlet" of each lenslet with the diffraciton limited psf of a lenslet. !!! -> more than good enough for now
    
    if SAVE:
        fits.writeto('data/ps_'+str(i)+'.fits', phase_screen[dimmat//2-PD[0]//2:dimmat//2+PD[0]//2,dimmat//2-PD[0]//2:dimmat//2+PD[0]//2], overwrite=True)
        fits.writeto('data/psf_'+str(i)+'.fits', psf, overwrite=True)
        fits.writeto('data/DwDx_'+str(i)+'.fits', DwDx, overwrite=True)
        fits.writeto('data/DwDy_'+str(i)+'.fits', DwDy, overwrite=True)
        fits.writeto('data/lightfield_'+str(i)+'.fits', S*ows.normalize(lightfield), overwrite=True)

    if 100*i//n == status*10:
        now = time.time()
        elapsed = now - before
        if status != 0:
            print('Processing: ',status*10,'% done, time elapsed: ',elapsed,'s, remaining: ~',((elapsed/(i/n))-elapsed),'s')
        status += 1
psf_stack = psf_stack/n
end = time.time()
print(psf_stack.max())

print("Processing time :",
      (end-start) * 10**3, "ms")
print("S: ",S)
plt.close(3)
plt.figure(3)
plt.imshow(psf)
plt.title("last phase screen")
plt.show()

plt.close(4)
plt.figure(4)
plt.plot(psf[32,:])
plt.plot(psf_stack[32,:])
plt.axhline(y=psf_stack.max()/2,c='k')
plt.ylim([0,1])
plt.legend(["last psf",f"'long' exposure psf({n:0} instantaneous)"])
plt.title("Generated PSF")


# Other data that could be displayed

# plt.close(5)
# plt.figure(5)
# plt.subplot(1,3,1)
# plt.imshow(lightfield)
# plt.title("last lightfield")
# plt.subplot(1,3,2)
# plt.imshow(DwDx)
# plt.title("last DwDx")
# plt.subplot(1,3,3)
# plt.imshow(DwDy)
# plt.title("last DwDy")

# plt.close(5)
# plt.figure(5)
# plt.imshow(lightfield)
# plt.colorbar()
# plt.title("Example of instantaneous lightfield")

# plt.close(6)
# plt.figure(6)
# plt.imshow(DwDx)
# plt.title("DwDx associated to the lightfield")
# plt.colorbar()
# plt.close(7)
# plt.figure(7)

# plt.imshow(DwDy)
# plt.colorbar()
# plt.title("DwDy associated to the lightfield")

plt.show()


# <a id='toc3_'></a>[Long exposure PSF](#toc0_)

In [ ]:
dimmat_psf =64
delta_nu = 2*D/(dimmat_psf*wl)  # Angular frequency sampling
nu_x = np.linspace(-dimmat_psf/2, dimmat_psf/2-1, dimmat_psf) * delta_nu
NU_X, NU_Y = np.meshgrid(nu_x, nu_x)
nu = np.sqrt(NU_X**2 + NU_Y**2)
nu_n = nu * wl / D

otf_tsc = ows.telescope_otf(nu_n)
otf_atm = ows.atmospheric_otf(nu, r0, L0, wl)

otf_sl = otf_atm * otf_tsc

strehl = np.sum(otf_sl) / np.sum(otf_tsc)

padding = 2
otf_sl_pad = np.zeros((padding*dimmat_psf,padding*dimmat_psf))
otf_sl_pad[0:dimmat_psf,0:dimmat_psf] = otf_sl
longpsf = np.abs(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(otf_sl_pad))))[dimmat_psf-dimmat_psf//2:dimmat_psf+dimmat_psf//2,dimmat_psf-dimmat_psf//2:dimmat_psf+dimmat_psf//2]
print(longpsf.sum())
longpsf = ows.normalize(longpsf)
longpsf = longpsf*strehl

otf_tsc_pad = np.zeros((padding*dimmat_psf,padding*dimmat_psf))
otf_tsc_pad[0:dimmat_psf,0:dimmat_psf] = otf_tsc
idealpsf = np.abs(np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(otf_tsc_pad))))[dimmat_psf-dimmat_psf//2:dimmat_psf+dimmat_psf//2,dimmat_psf-dimmat_psf//2:dimmat_psf+dimmat_psf//2]

idealpsf = ows.normalize(idealpsf)

print(strehl)

plt.close(1)
plt.figure(1)
# plt.subplot(1,2,1)
plt.plot(longpsf[32,:])
plt.axhline(y=longpsf.max()/2,c='k')
plt.title("Long exposure psf")
plt.ylim([0,1])
plt.xlim([0,dimmat_psf])
text = "S: "+str(strehl)
print(text)
plt.text(10,.8,text)
# plt.colorbar()

# plt.subplot(1,2,2)
# plt.imshow(otf_atm)

plt.show()



# fits.writeto('data/longpsf.fits', longpsf, overwrite=True)


# <a id='toc4_'></a>[Lenslet array illumination](#toc0_)

# <a id='toc5_'></a>[SHWFS](#toc0_)

## <a id='toc5_1_'></a>[Lenslet illumination mask](#toc0_)

Lenslet with coverage $\geq$ 50\% are considered.

## <a id='toc5_2_'></a>[Wavefront slope](#toc0_)


1. Focal plane image:
$$U_f= \frac{1}{i\lambda F}\mathcal{F}\{t_p(x,y)\}$$

where $t_p(x,y)$ is the object (the phase screen in this case)


2. Compute the spot centers

$$x_{c,k} = \frac{\sum_{i\in k}\sum_{j\in k} x_{i,j}I_{i,j}}{\sum_{i\in k}\sum_{j\in k} I_{i,j}}$$

$$y_{c,k} = \frac{\sum_{i\in k}\sum_{j \in k} y_{i,j}I_{i,j}}{\sum_{i\in k}\sum_{j\in k} I_{i,j}}$$

3. Extract the WF slopes

$$ \begin{bmatrix}\partial w /\partial x \\ \partial w/\partial y\end{bmatrix}_k = \begin{bmatrix} \beta_x\\\beta_y \end{bmatrix}_k \approx \frac{1}{L_H}\begin{bmatrix} x_c - x_r\\y_c - y_r \end{bmatrix}_k$$
